In [6]:
import requests
import os
from io import BytesIO
from zipfile import ZipFile

In [2]:
# Official page: https://gtfs.adelaidemetro.com.au/
# URLs
base_url = "http://gtfs.adelaidemetro.com.au/v1"
latest_version_number = "static/latest/version.txt"
latest_version_feed = "static/latest/google_transit.zip"

# binary data as response
request_headers={"Content-Type": "application/octet-stream"}

# Feed template
# practice: how to use string template + format
feed_url_template = "static/{version}/google_transit.zip"

# output
dest_path = 'destination'
os.makedirs(dest_path, exist_ok=True)

Read latest version

In [ ]:
# 1. Find the latest feed version
# r = requests.get(f'{base_url}/{latest_version_number}')
# print(r.content)

# 2. Download latest GTFS feed version using request
r = requests.get(f'{base_url}/{latest_version_feed}',headers=request_headers)
with open(f'{dest_path}/latest.zip','wb') as f:
    for chunk in r.iter_content(chunk_size=128):
        f.write(chunk)
# # Feed's URL

Dynamic read multiple feed versions: 930 to 939

In [18]:
# HINTS
# 1. For loop the versions
# 2. Generate versioned URL
# 3. Request and handle response

for version in list(range(930,939)):
    # print(f'{base_url}/static/{version}/google_transit.zip')
    r = requests.get(f'{base_url}/static/{version}/google_transit.zip',headers=request_headers)
    
    with open(f'{version}.zip','wb') as f:
        for chunk in r.iter_content(chunk_size=128):
            f.write(chunk)

Unzip feed versions & export files

In [8]:
# # using ZipFile to read zipfile
# # then extract all

# # Define the path to your zip file
zip_file_path = 'destination/latest.zip' 

# # Define the directory where you want to extract the files (optional)
# # If not specified, files will be extracted to the current working directory
extract_to_path = 'destination/latest/' 
os.makedirs(extract_to_path,exist_ok=True)

with ZipFile(zip_file_path, 'r') as zip_obj:
    # Extract all files to the specified directory
    zip_obj.extractall(extract_to_path)

Extra: extract zipfile from response.content

In [ ]:
# HINTS
# Use BytesIO (file bytes in memory) + ZipFile